In this notebook, I am trying to demonstrate that some values in the output produced by INPHARED with regard to the Genbankfile it processes with its inphared.pl script is not correct

In [5]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqFeature import FeatureLocation
from Bio.Seq import UndefinedSequenceError

The input is a folder with a pharokka output. We will parse several files and compare them to prove that the output of INPHARED is not quite right

In [2]:
code = "MZ079855__Klebsiella_phage_vB_Kpn_3" 
entry = f"./{code}.fna_pharokka/"
entry

'./MZ079855__Klebsiella_phage_vB_Kpn_3.fna_pharokka/'

## Parsing of Genbank file by custom Python function

This function extracts information from a genbank file with a similar logic to the inphared.pl script. We will focus on the GC content, number of CDS, positive and negative %, and coding capacity

In [3]:
genbank_file_path = f'{entry}pharokka.gbk'

def engineer_features(genbank_file):
    # Lists to hold data
    ids = []
    genome_lengths = []
    gc_contents = []
    sequences = []
    reverse_complements = []
    cds_numbers = []
    positive_strands = []
    negative_strands = []
    coding_capacities = []
    molecule_types = []
    topologies = []
    trna_counts = []

    
    # Read the GenBank file
    for record in SeqIO.parse(genbank_file, "genbank"):
        try:
            # Attempt to access the sequence, which may raise UndefinedSequenceError
            sequence = str(record.seq)
            # print(record.id)
        except UndefinedSequenceError:
            # print(f"Skipping record {record.id} as sequence is undefined.")
            continue  # Skip this record

        # Calculate genome length and GC content
        total_length = len(sequence)
        gc_content = round((sequence.count('C') + sequence.count('G')) / total_length * 100, 3)

        # Initialize counters
        plus = 0
        minus = 0
        coding_count = 0
        trna_count = 0
        seen = set()  # Store seen barcodes

        for feature in record.features:
            start = feature.location.start
            end = feature.location.end
            length = len(FeatureLocation(start, end))
            barcode = f"{start}_{end}_{length}"

            if feature.type != 'source' and barcode not in seen:
                coding_count += length
                seen.add(barcode)

            if feature.type == 'CDS':
                if feature.location.strand == 1:
                    plus += 1
                elif feature.location.strand == -1:
                    minus += 1
            elif feature.type == 'tRNA':
                trna_count += 1

        
        # Calculate total number of CDS
        total_CDS = plus + minus

        # Calculate strand usage as a percentage
        per_plus = round((plus / total_CDS) * 100, 2) if total_CDS != 0 else 0
        per_minus = round((minus / total_CDS) * 100, 2) if total_CDS != 0 else 0

        # Calculate coding capacity as a percentage
        coding_capacity = (coding_count / total_length) * 100

        # Extract molecule_type and topology
        molecule_type = record.annotations.get('molecule_type', 'N/A')
        topology = record.annotations.get('topology', 'N/A')

        # Append data to lists
        ids.append(record.id)
        genome_lengths.append(total_length)
        gc_contents.append(gc_content)
        sequences.append(sequence)
        reverse_complements.append(str(sequence[::-1]))
        cds_numbers.append(total_CDS)
        positive_strands.append(per_plus)
        negative_strands.append(per_minus)
        coding_capacities.append(coding_capacity)
        molecule_types.append(molecule_type)
        topologies.append(topology)
        trna_counts.append(trna_count)
    
    print("Processing the entries...")
    # Convert lists to pandas DataFrame
    df = pd.DataFrame({
        'id': ids,
        'genome_length': genome_lengths,
        'gc_%': gc_contents,
        'sequence': sequences,
        'reverse_complement': reverse_complements,
        'cds_number': cds_numbers,
        'positive_strand_%': positive_strands,
        'negative_strand_%': negative_strands,
        'coding_capacity': coding_capacities,
        'molecule_type': molecule_types,
        'topology': topologies,
        'trna_count': trna_counts
    })


    df['id'] = df['id'].str[:-2]

    # Check for unexpected molecule types
    expected_molecule_types = ['ss-DNA', 'DNA', 'RNA', 'ss-RNA']

    # Check and correct 'cRNA' entries
    cRNA_entries = df[df['molecule_type'] == 'cRNA']
    if not cRNA_entries.empty:
        for entry_id in cRNA_entries['id']:
            print(f"Info: Entry with id '{entry_id}' has molecule type 'cRNA'. Changing it to 'RNA'.")
        df.loc[df['molecule_type'] == 'cRNA', 'molecule_type'] = 'RNA'

    # Check and correct 'cDNA' entries
    cDNA_entries = df[df['molecule_type'] == 'cDNA']
    if not cDNA_entries.empty:
        for entry_id in cDNA_entries['id']:
            print(f"Info: Entry with id '{entry_id}' has molecule type 'cDNA'. Changing it to 'DNA'.")
        df.loc[df['molecule_type'] == 'cDNA', 'molecule_type'] = 'DNA'

    unexpected_types = set(df['molecule_type']) - set(expected_molecule_types)

    if unexpected_types:
        for utype in unexpected_types:
            # Get ids of entries with the unexpected molecule type
            ids_to_exclude = df[df['molecule_type'] == utype]['id'].tolist()
            for entry_id in ids_to_exclude:
                print(f"Warning: Entry with id '{entry_id}' has unrecognized molecule type '{utype}'. It will not be considered.")
            df = df[df['molecule_type'] != utype]
            
    df = pd.get_dummies(df, columns=['molecule_type'])

    expected_columns = ['jumbophage', 'molecule_type_ss-DNA', 'molecule_type_DNA', 'molecule_type_RNA', 'molecule_type_ss-RNA']
    for col in expected_columns:
        if col not in df.columns:
            df[col] = 0  # Filling with zeros
        df[col] = df[col].astype(bool)  # Convert to boolean

    df['jumbophage'] = df['genome_length'].apply(lambda x: x >= 200000)
    df['jumbophage'] = df['jumbophage'].astype(int)  # Convert True/False to 1/0

    return df

df = engineer_features(genbank_file_path)


feature_columns = ['id', 'genome_length', 'gc_%',
       'cds_number', 'positive_strand_%', 'negative_strand_%',
       'coding_capacity',  'trna_count']

print("---Information gathered by my function----")
for column in df.columns:
    if column in feature_columns:
        print(f"Column {column}: ",df[column].to_list())




FileNotFoundError: [Errno 2] No such file or directory: './MZ079855__Klebsiella_phage_vB_Kpn_3.fna_pharokka/pharokka.gbk'

## Parsing of Genbank file by INPHARED

We are now going to load the same information coming from the file produced by inphared. This has been presumely produced with the inphared.pl as part of the pharokka kit

In [ ]:
df_inphared = pd.read_csv(f"{entry}pharokka_top_hits_mash_inphared.tsv", sep="\t")

feature_columns = ['Accession', 
       'Genome_Length_(bp)',  'molGC_(%)', 
        'Number_CDS', 'Positive_Strand_(%)',
       'Negative_Strand_(%)', 'Coding_Capacity_(%)',
       'tRNAs']
print("---Information gathered by INPHARED---")
for column in df_inphared.columns:
    if column in feature_columns:
        print(f"Column {column}: ",df_inphared[column].to_list())


---Information gathered by INPHARED---
Column Accession:  ['MZ079855']
Column Genome_Length_(bp):  [112003]
Column molGC_(%):  [41.361]
Column Number_CDS:  [182]
Column Positive_Strand_(%):  [70.8791208791209]
Column Negative_Strand_(%):  [29.1208791208791]
Column Coding_Capacity_(%):  [83.2638411471121]
Column tRNAs:  [19]


In [ ]:
df_inphared = pd.read_csv(f"{entry}pharokka_top_hits_mash_inphared.tsv", sep="\t")

feature_columns = ['Accession', 
       'Genome_Length_(bp)',  'molGC_(%)', 
        'Number_CDS', 'Positive_Strand_(%)',
       'Negative_Strand_(%)', 'Coding_Capacity_(%)',
       'tRNAs']
print("---Information gathered by INPHARED---")
for column in df_inphared.columns:
    if column in feature_columns:
        print(f"Column {column}: ",df_inphared[column].to_list())


In [4]:
def engineer_features(genbank_file):
    # Lists to hold data
    ids = []
    genome_lengths = []
    gc_contents = []
    sequences = []
    reverse_complements = []
    cds_numbers = []
    positive_strands = []
    negative_strands = []
    coding_capacities = []
    molecule_types = []
    topologies = []
    trna_counts = []

    
    # Read the GenBank file
    for record in SeqIO.parse(genbank_file, "genbank"):
        try:
            # Attempt to access the sequence, which may raise UndefinedSequenceError
            sequence = str(record.seq)
            # print(record.id)
        except UndefinedSequenceError:
            # print(f"Skipping record {record.id} as sequence is undefined.")
            continue  # Skip this record

        # Calculate genome length and GC content
        total_length = len(sequence)
        gc_content = round((sequence.count('C') + sequence.count('G')) / total_length * 100, 3)

        # Initialize counters
        plus = 0
        minus = 0
        coding_count = 0
        trna_count = 0
        seen = set()  # Store seen barcodes

        for feature in record.features:
            start = feature.location.start
            end = feature.location.end
            length = len(FeatureLocation(start, end))
            barcode = f"{start}_{end}_{length}"

            if feature.type != 'source' and barcode not in seen:
                coding_count += length
                seen.add(barcode)

            if feature.type == 'CDS':
                if feature.location.strand == 1:
                    plus += 1
                elif feature.location.strand == -1:
                    minus += 1
            elif feature.type == 'tRNA':
                trna_count += 1

        
        # Calculate total number of CDS
        total_CDS = plus + minus

        # Calculate strand usage as a percentage
        per_plus = round((plus / total_CDS) * 100, 2) if total_CDS != 0 else 0
        per_minus = round((minus / total_CDS) * 100, 2) if total_CDS != 0 else 0

        # Calculate coding capacity as a percentage
        coding_capacity = (coding_count / total_length) * 100

        # Extract molecule_type and topology
        molecule_type = record.annotations.get('molecule_type', 'N/A')
        topology = record.annotations.get('topology', 'N/A')

        # Append data to lists
        ids.append(record.id)
        genome_lengths.append(total_length)
        gc_contents.append(gc_content)
        sequences.append(sequence)
        reverse_complements.append(str(sequence[::-1]))
        cds_numbers.append(total_CDS)
        positive_strands.append(per_plus)
        negative_strands.append(per_minus)
        coding_capacities.append(coding_capacity)
        molecule_types.append(molecule_type)
        topologies.append(topology)
        trna_counts.append(trna_count)
    
    print("Processing the entries...")
    # Convert lists to pandas DataFrame
    df = pd.DataFrame({
        'id': ids,
        'genome_length': genome_lengths,
        'gc_%': gc_contents,
        'sequence': sequences,
        'reverse_complement': reverse_complements,
        'cds_number': cds_numbers,
        'positive_strand_%': positive_strands,
        'negative_strand_%': negative_strands,
        'coding_capacity': coding_capacities,
        'molecule_type': molecule_types,
        'topology': topologies,
        'trna_count': trna_counts
    })


    df['id'] = df['id'].str[:-2]

    # Check for unexpected molecule types
    expected_molecule_types = ['ss-DNA', 'DNA', 'RNA', 'ss-RNA']

    # Check and correct 'cRNA' entries
    cRNA_entries = df[df['molecule_type'] == 'cRNA']
    if not cRNA_entries.empty:
        for entry_id in cRNA_entries['id']:
            print(f"Info: Entry with id '{entry_id}' has molecule type 'cRNA'. Changing it to 'RNA'.")
        df.loc[df['molecule_type'] == 'cRNA', 'molecule_type'] = 'RNA'

    # Check and correct 'cDNA' entries
    cDNA_entries = df[df['molecule_type'] == 'cDNA']
    if not cDNA_entries.empty:
        for entry_id in cDNA_entries['id']:
            print(f"Info: Entry with id '{entry_id}' has molecule type 'cDNA'. Changing it to 'DNA'.")
        df.loc[df['molecule_type'] == 'cDNA', 'molecule_type'] = 'DNA'

    unexpected_types = set(df['molecule_type']) - set(expected_molecule_types)

    if unexpected_types:
        for utype in unexpected_types:
            # Get ids of entries with the unexpected molecule type
            ids_to_exclude = df[df['molecule_type'] == utype]['id'].tolist()
            for entry_id in ids_to_exclude:
                print(f"Warning: Entry with id '{entry_id}' has unrecognized molecule type '{utype}'. It will not be considered.")
            df = df[df['molecule_type'] != utype]
            
    df = pd.get_dummies(df, columns=['molecule_type'])

    expected_columns = ['jumbophage', 'molecule_type_ss-DNA', 'molecule_type_DNA', 'molecule_type_RNA', 'molecule_type_ss-RNA']
    for col in expected_columns:
        if col not in df.columns:
            df[col] = 0  # Filling with zeros
        df[col] = df[col].astype(bool)  # Convert to boolean

    df['jumbophage'] = df['genome_length'].apply(lambda x: x >= 200000)
    df['jumbophage'] = df['jumbophage'].astype(int)  # Convert True/False to 1/0
    df = pd.get_dummies(df, columns=['topology'])
    return df


In [6]:
engineer_features("/Users/Alvaro/Library/Mobile Documents/com~apple~CloudDocs/DTU/Autumn 2023/Phage project/phage/data/raw/GenomesDB/MZ079855/MZ079855.gbf")

Processing the entries...


,id,genome_length,gc_%,sequence,reverse_complement,cds_number,positive_strand_%,negative_strand_%,coding_capacity,trna_count,molecule_type_DNA,jumbophage,molecule_type_ss-DNA,molecule_type_RNA,molecule_type_ss-RNA,topology_linear
0,MZ0798,112003,41.361,TGTCACCGTGAGTTCGCTTGTTTTTTAGAGCCACAGGAATTTTTTC...,TTTTAAAATAATCGTTTTACCCCGTTTTATCGAACTGTTCACTTTT...,182,70.88,29.12,83.263841,19,True,0,False,False,False,True


### What are we looking at?

There are **two conflicting elements**:
- Number of CDS: 186 (mine) vs 182 (inphared)
- Number of tRNAs: 17 (mine) vs 19 (inphared)


The coding capacity, and +/- % also differs, but that is probably related to the fact that the number of CDS is different.


## Checking the correct numbers


We can check in other pharokka files that the correct number is that of my function, and therefore there is something not being parsed correctly in inphared.

**Number of CDS and tRNAs**

In [ ]:
df = pd.read_csv(f"{entry}pharokka_cds_functions.tsv", sep="\t")
df.iloc[[0,11]]

,Description,Count,contig
0,CDS,186,MZ079855__Klebsiella_phage_vB_Kpn_3
11,tRNAs,23,MZ079855__Klebsiella_phage_vB_Kpn_3


- The number of CDS is 186 and, thus, inphared.pl misses 4 of them.
- Both my function and inphared return the wrong number of tRNAs. If we look at the Genbank file, we will see the reason for this. 

    tRNAs can be stored in the genbank file either as features of their own (which in this file they sum up to 17) or as part of pseudogenes (which they sum up to the value offered in this other file: 23). Whichever the case, it seems that inphared is not parsing the file correctly

**Coding density**

In [25]:
df = pd.read_csv(f"{entry}pharokka_length_gc_cds_density.tsv", sep="\t")
for column in df:
    if column == "cds_coding_density":
        print(f"Column {column}: ",df[column].to_list())


Column cds_coding_density:  [82.86]


In this case, the coding density is slightly different to both my function and inphared, and I have not found out why. 